In [ ]:
import pandas as pd
import sqlite3

In [ ]:
df = pd.read_csv('/kaggle/input/spotify-top-50-songs-in-2021/spotify_top50_2021.csv', index_col=[0])

df.head()

In [ ]:
#Connect to database
conn = sqlite3.connect('database.db')
c = conn.cursor()

In [ ]:
df.to_sql("TopSongs", conn)

In [ ]:
#Query table

c.execute('''SELECT * FROM TopSongs;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

# Artist Name
Lets take a look at the artists that have songs in the top 50 of 2021


In [ ]:
#Query the artists in top 50 and the count of top songs by each artist
c.execute('''SELECT artist_name, COUNT(*)
			 FROM TopSongs
             GROUP BY artist_name
             ORDER BY COUNT(*) DESC;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

We can see Olivia Rodrigo and Doja Cat were the only artists who have 4 songs in the top 50 of 2021. 10 artists total have 2 or more than 2 songs in the top 50, everyone else has only 1 song.

In [ ]:
#Query the songs that have another artist featured on them

c.execute('''SELECT track_name
			 FROM TopSongs
             WHERE track_name LIKE '%feat%' OR track_name LIKE '%ft.%' OR track_name LIKE '%(with%';''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data


In [ ]:
#Query the number of songs that have another artist featured on them

c.execute('''SELECT COUNT(*)
			 FROM TopSongs
             WHERE track_name LIKE '%feat%' OR track_name LIKE '%ft.%' OR track_name LIKE '%(with%';''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

9 of the top 50 songs featured another artist, that is 18% of all the songs. Perhaps having a featured artist on a track helps a song end up on the Spotify Top 50.

# Energy
Next lets take a look at the energy of the songs in Spotify Top 50

In [ ]:
# Query the Average Energy from TopSongs

c.execute('''SELECT ROUND(AVG(energy), 3) as average_energy
			 FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

The average energy for the songs in Spotify Top 50 is 0.646. Lets see how all the songs compare to the average.

In [ ]:
c.execute('''SELECT track_name, artist_name, energy,
			 CASE
             WHEN energy > 0.646 THEN "Above average"
             WHEN energy = 0.646 THEN "Average"
             WHEN energy < 0.646 THEN "Below average"
             END AS compareEnergy
             FROM TopSongs
             LIMIT 25;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

Only 8 of the top 25 tracks were below average. This suggests that songs with more energy are more likely to be listed within the Top 50 on Spotify.

# Danceability
Next, taking a look at danceability.

In [ ]:
#Query the average Danceablity in TopSonfs

c.execute('''SELECT ROUND(AVG(danceability), 3) as avg_danceability
			 FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

The average danceability of songs in this data set in 0.686. The code below labels each song as to how close their danceability score is to the average.

In [ ]:
# Multiplying the danceablity parameter by 100 just to make comparison easier

c.execute('''SELECT track_name, danceability*100 as danceability,
			 CASE
             WHEN (danceability*100 BETWEEN 58.5 AND 78.6) THEN "within 10"
             WHEN (danceability*100 BETWEEN 48.6 AND 58.5) OR (danceability*100 BETWEEN 78.7 AND 88.6) THEN "within 20"
             WHEN (danceability*100 BETWEEN 38.6 AND 48.5) OR (danceability*100 BETWEEN 88.7 AND 98.6) THEN "within 30"
             ELSE "greater than 31" 
             END AS compare_avg
             FROM TopSongs
             LIMIT 15;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

In [ ]:
# Multiplying the danceablity parameter by 100 just to make comparison easier

c.execute('''SELECT 
			 SUM(
             CASE
             WHEN (danceability*100 BETWEEN 58.5 AND 78.6) THEN 1 
             ELSE 0
             END) as Within10,
             SUM(
             CASE
             WHEN (danceability*100 BETWEEN 48.6 AND 58.5) OR (danceability*100 BETWEEN 78.7 AND 88.6) THEN 1
             ELSE 0
             END) as Within20,
             SUM(
             CASE
             WHEN (danceability*100 BETWEEN 38.6 AND 48.5) OR (danceability*100 BETWEEN 88.7 AND 98.6) THEN 1
             ELSE 0
             END) as Within30,
             SUM(
             CASE
             WHEN (danceability*100 < 38.6) OR (danceability*100 > 98.6) THEN 1
             ELSE 0
             END) as GreaterThan30 
             FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

The average danceability score for the entire table, which is 68.6, is within 10 points of what 58% of the songs in the dataset achieve. It seems that a song's chances of making the Spotify top 50 are increased if its danceability score is near to 68.6.

# Loudness
Continuing with Loudness as the next parameter.

In [ ]:
#Query the Average Loudness from TopSongs

c.execute('''SELECT ROUND(AVG(loudness), 3) as avg_loudness
			 FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

In [ ]:
# examining the top ten songs in relation to this average 

c.execute('''SELECT track_name, loudness,
			 CASE
             WHEN loudness < -5.904 THEN "Below average"
             WHEN loudness = -5.904 THEN "Average"
             WHEN loudness > -5.904 THEN "Above average"
             END AS compare_avg
             FROM TopSongs
             LIMIT 10;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

In [ ]:
c.execute('''SELECT track_name, loudness,
			 CASE
             WHEN (loudness BETWEEN -6.904 AND -4.903) THEN "within 1"
             WHEN (loudness BETWEEN -4.904 AND -3.903) OR (loudness BETWEEN -7.904 AND -6.903) THEN "within 2"
             WHEN (loudness BETWEEN -3.904 AND -2.903) OR (loudness BETWEEN -8.904 AND -7.903) THEN "within 3"
             ELSE "greater than 3" 
             END AS compare_avg
             FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

In [ ]:
c.execute('''SELECT 
			 SUM(
             CASE
             WHEN (loudness BETWEEN -6.904 AND -4.903) THEN 1 
             ELSE 0
             END) as Within1,
             SUM(
             CASE
             WHEN (loudness BETWEEN -4.904 AND -3.903) OR (loudness BETWEEN -7.904 AND -6.903) THEN 1
             ELSE 0
             END) as Within2,
             SUM(
             CASE
             WHEN (loudness BETWEEN -3.904 AND -2.903) OR (loudness BETWEEN -8.904 AND -7.903) THEN 1
             ELSE 0
             END) as Within3,
             SUM(
             CASE
             WHEN (loudness > -2.904) OR (loudness < -8.904) THEN 1
             ELSE 0
             END) as GreaterThan3 
             FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

# Speechiness

#### Spotify defines Speechiness as follows:
Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. 

In [ ]:
# Query the average speechiness in TopSongs

c.execute('''SELECT AVG(speechiness) as avg_speechiness
			 FROM TopSongs;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

At the time this dataset was created, the average Speechiness of the top 50 Spotify Songs was 0.08. This is less than the 0.33 that Spotify recommends for most songs.

In [ ]:
# Query the songs with highest speechiness

c.execute('''SELECT track_name, artist_name, speechiness
			 FROM TopSongs
             ORDER BY speechiness DESC
             LIMIT 5;''')

data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

The tracks with the highest speechiness can frequently be categorized as "Rap," although being much below the advised 0.33, in Spotify Top 50.